In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1618791/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_1618791/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_3_3_0,0.507012,-0.323765,-1.118870,-0.264273,0.470905,2.653738,1.562482,1.155893,5.308090e+13,0.686225,-1.103414,0.697032,99.506198,206.051175,"Hidden Size=[12], regularizer=0.02, learning_r..."
1,model_3_9_7,0.716151,-0.345616,-1.698516,0.645978,0.271134,2.697542,1.989921,0.822082,1.531952e+14,0.520705,-0.211088,0.528906,100.610284,207.155260,"Hidden Size=[12], regularizer=0.02, learning_r..."
2,model_4_15_7,0.758657,-0.380667,-0.703934,0.892769,0.230533,2.767810,1.256503,0.208565,7.457851e+13,0.480138,1.468060,0.487700,196.934726,407.850291,"Hidden Size=[24], regularizer=0.02, learning_r..."
3,model_4_16_2,0.764529,-0.384283,-0.692533,0.306507,0.224923,2.775057,1.248095,0.504016,7.412768e+13,0.474260,1.456670,0.481730,196.983995,407.899560,"Hidden Size=[24], regularizer=0.02, learning_r..."
4,model_4_13_7,0.656794,-0.386790,-0.932231,-0.016193,0.327832,2.780084,1.424852,0.289170,7.525642e+13,0.572566,1.665611,0.581584,196.230506,407.146071,"Hidden Size=[24], regularizer=0.02, learning_r..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,model_3_48_1,0.679858,-86.293438,-1.862002,0.217819,0.305802,174.996252,2.110477,0.576975,1.010249e+14,0.552994,-0.365941,0.561703,100.369635,206.914611,"Hidden Size=[12], regularizer=0.02, learning_r..."
132,model_3_47_3,0.677551,-90.777838,-2.038265,0.350505,0.308006,183.986081,2.240456,0.504989,1.386243e+14,0.554983,-0.375784,0.563723,100.355275,206.900251,"Hidden Size=[12], regularizer=0.02, learning_r..."
133,model_3_48_7,0.688877,-102.303952,-1.742999,0.040909,0.297186,207.092363,2.022723,0.707472,9.948358e+13,0.545148,-0.327458,0.553734,100.426791,206.971767,"Hidden Size=[12], regularizer=0.02, learning_r..."
134,model_3_49_0,0.692863,-106.336724,-1.731516,-0.229411,0.293379,215.176819,2.014255,0.655278,9.333795e+13,0.541645,-0.310451,0.550175,100.452579,206.997555,"Hidden Size=[12], regularizer=0.02, learning_r..."
